In [1]:
import numpy as np
import csv
import pickle
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Import the original dataset

Load data and remove unnecessary columns

In [2]:
### Change the name of the csv file if needed

file_name = "dataset.csv"

df = pd.read_csv("local_data/"+file_name)
# df = df.drop(columns=['id_submission','submissiontype.co','bounced'])
df = df.drop(index = df.loc[df['not_commenter']==0].index)

df.reset_index(drop=True, inplace=True)

In [3]:
# df.columns

Extracting the target feature to be predicted

In [4]:
y = df['not_commenter']
df = df.drop(columns=['not_commenter'])

In [5]:
# df.shape

Get the one-hot vector for campaigns

In [6]:
campaigns = []
for name in df.columns:
    if "campaign." in name:
        campaigns.append(name)

campaigns_df = df[campaigns]
df = df.drop(columns=campaigns)

In [7]:
campaigns_df

,campaign.-1,campaign.0,campaign.1,campaign.11,campaign.12,campaign.13,campaign.14,campaign.15,campaign.16,campaign.19,...,campaign.5,campaign.53,campaign.54,campaign.55,campaign.56,campaign.57,campaign.6,campaign.7,campaign.8,campaign.9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12813,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12814,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12815,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12816,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
words = []
for w in df.columns:
    if "text." in w:
        words.append(w)

bow_df = df[words]
df = df.drop(columns=words)

In [9]:
bow_df

,text.ability,text.access,text.back,text.bad,text.balance,text.best,text.big,text.block,text.business,text.certain,...,text.transform,text.trump,text.turn,text.urge,text.veto,text.want,text.way,text.well,text.wheeler,text.would
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12813,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12815,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
city = []
for c in df.columns:
    if "city_state." in c:
        city.append(c)

city_df = df[city]
df = df.drop(columns=city)

In [11]:
city_df

,city_state.EMPTY,city_state.INVALID,city_state.MISSING,city_state.abbeville sc,city_state.aberdeen sd,city_state.aberdeen wa,city_state.abernathy tx,city_state.abilene tx,city_state.abingdon md,city_state.abingdon va,...,city_state.yuba city ca,city_state.yucaipa ca,city_state.yucca valley ca,city_state.yuma az,city_state.zanesville oh,city_state.zebulon nc,city_state.zelienople pa,city_state.zephyrhills fl,city_state.zimmerman mn,city_state.zionsville in
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12813,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df['email_hash']=df['email_hash'].replace(df['email_hash'].value_counts().to_dict())

In [13]:
df

,all_centered_bin,all_submitted_bin,campaign_centered_bin,campaign_submitted_bin,date_disseminated,date_received,email_confirmation,email_hash,id_submission,submitted
0,250540,225539,77,67,1.503417e+09,1.502810e+09,0.0,1,1910822472124578,1.502810e+09
1,27023,27033,5,5,1.502393e+09,1.501877e+09,0.0,1,1910810001827427,1.501877e+09
2,229289,329274,31,31,1.502394e+09,1.502142e+09,0.0,1,19108101689102729,1.502142e+09
3,229289,329274,31,31,1.502394e+09,1.502142e+09,0.0,1,1910810098691473,1.502142e+09
4,201041,201041,43,43,1.503416e+09,1.502468e+09,0.0,1,1910822969918119,1.502468e+09
...,...,...,...,...,...,...,...,...,...,...
12813,2962,2951,2,1,1.494344e+09,1.494335e+09,1.0,1,105092134009937,1.494308e+09
12814,5753,5682,6,6,1.494270e+09,1.494262e+09,0.0,1,105080571410134,1.494262e+09
12815,71,69,1,1,1.495134e+09,1.495131e+09,0.0,1,10518247320620,1.495131e+09
12816,617,621,1,1,1.494780e+09,1.494437e+09,0.0,1,1051004758300,1.494437e+09


### Variance analysis

In [14]:
normalized_df=(df-df.min())/(df.max()-df.min())
v = normalized_df.var().sort_values(ascending=False)
v.sort_values(ascending=False)

campaign_submitted_bin    0.142463
campaign_centered_bin     0.142420
all_submitted_bin         0.037280
date_received             0.034535
submitted                 0.034518
email_confirmation        0.033681
date_disseminated         0.032552
all_centered_bin          0.025313
email_hash                0.016926
id_submission             0.007317
dtype: float64

In [15]:
v_campaigns = campaigns_df.var().sort_values(ascending=False)
v_campaigns.nlargest(10)

campaign.29    0.105658
campaign.-1    0.104946
campaign.13    0.099426
campaign.35    0.089332
campaign.38    0.086631
campaign.56    0.084861
campaign.30    0.080520
campaign.57    0.065825
campaign.36    0.028255
campaign.1     0.021220
dtype: float64

In [16]:
v_city = city_df.var().sort_values(ascending=False)
v_city.nlargest(10)

city_state.EMPTY               0.107549
city_state.MISSING             0.011871
city_state.chicago il          0.011642
city_state.new york ny         0.011490
city_state.los angeles ca      0.009811
city_state.brooklyn ny         0.009658
city_state.seattle wa          0.009428
city_state.portland or         0.008662
city_state.san francisco ca    0.008432
city_state.philadelphia pa     0.005277
dtype: float64

In [17]:
normalized_bow=(bow_df-bow_df.min())/(bow_df.max()-bow_df.min())
v_bow = normalized_bow.var().sort_values(ascending=False)
v_bow.sort_values(ascending=False)

text.bad           0.086844
text.transform     0.055780
text.current       0.053410
text.block         0.046948
text.stand         0.045900
                     ...   
text.technology    0.000986
text.nation        0.000648
text.service       0.000594
text.people        0.000578
text.sincerely     0.000312
Length: 100, dtype: float64

In [18]:
v = v.to_frame(name="Var")
v_city = v_city.to_frame(name="Var")
v_bow = v_bow.to_frame(name="Var")
v_campaigns = v_campaigns.to_frame(name="Var")
v.to_csv('var/V_Norm.csv')
v_city.to_csv('var/V_City.csv')
v_bow.to_csv('var/V_BOW.csv')
v_campaigns.to_csv('var/V_Campaigns.csv')
normalized_df.to_csv('df/Norm.csv')
city_df.to_csv('df/City.csv')
normalized_bow.to_csv('df/BOW.csv')
campaigns_df.to_csv('df/Campaigns.csv')

NameError: name 'city_state_df' is not defined

In [19]:
city_df.to_csv('df/City.csv')
normalized_bow.to_csv('df/BOW.csv')
campaigns_df.to_csv('df/Campaigns.csv')

In [20]:
# thr = 0.001
# min_var = thr*(1-thr)
# city_state_df_thr = city_state_df[v_city_state.loc[v_city_state['Var']>=thr].index.to_list()]

In [21]:
normalized_df

,all_centered_bin,all_submitted_bin,campaign_centered_bin,campaign_submitted_bin,date_disseminated,date_received,email_confirmation,email_hash,id_submission,submitted
0,0.608317,0.684957,0.051386,0.044625,0.644279,0.605636,0.5,0.0,0.099991,0.605636
1,0.065610,0.082096,0.002705,0.002705,0.578468,0.545663,0.5,0.0,0.099991,0.545663
2,0.556719,1.000000,0.020284,0.020284,0.578490,0.562720,0.5,0.0,0.999952,0.562720
3,0.556719,1.000000,0.020284,0.020284,0.578490,0.562720,0.5,0.0,0.099991,0.562720
4,0.488132,0.610557,0.028398,0.028398,0.644239,0.583652,0.5,0.0,0.099991,0.583652
...,...,...,...,...,...,...,...,...,...,...
12813,0.007189,0.008959,0.000676,0.000000,0.061024,0.060949,1.0,0.0,0.005494,0.059210
12814,0.013966,0.017253,0.003381,0.003381,0.056295,0.056279,0.5,0.0,0.005494,0.056279
12815,0.000170,0.000207,0.000000,0.000000,0.111833,0.112107,0.5,0.0,0.000545,0.112107
12816,0.001496,0.001883,0.000000,0.000000,0.089104,0.067470,0.5,0.0,0.000050,0.067470


In [22]:
baseline_df_nocity = pd.concat([normalized_df[['date_disseminated','date_received','submitted','email_confirmation']]],axis=1)
baseline_df_nocity.to_csv('db/BaseNoCity_v0.csv',index=False)

baseline_df = pd.concat([normalized_df[['date_disseminated','date_received','submitted','email_confirmation']],city_df],axis=1)
baseline_df.to_csv('db/Base_v0.csv',index=False)

baseline_cam_df = pd.concat([baseline_df,campaigns_df],axis=1)
baseline_df.to_csv('db/BaseCampaign_v0.csv',index=False)

baseline_cam_bins_df = pd.concat([baseline_cam_df,normalized_df[['campaign_centered_bin','campaign_submitted_bin']]],axis=1)
baseline_cam_bins_df.to_csv('db/BaseCampaignBin_v0.csv',index=False)

baseline_bow_df = pd.concat([baseline_df,normalized_bow],axis=1)
baseline_bow_df.to_csv('db/BaseBow_v0.csv',index=False)

baseline_bow_bins_df = pd.concat([baseline_bow_df,normalized_df[['all_centered_bin','all_submitted_bin']]],axis=1)
baseline_bow_bins_df.to_csv('db/BaseBowBin_v0.csv',index=False)

y.to_csv('db/Y.csv',index=False)

In [23]:
baseline_df

,date_disseminated,date_received,submitted,email_confirmation,city_state.EMPTY,city_state.INVALID,city_state.MISSING,city_state.abbeville sc,city_state.aberdeen sd,city_state.aberdeen wa,...,city_state.yuba city ca,city_state.yucaipa ca,city_state.yucca valley ca,city_state.yuma az,city_state.zanesville oh,city_state.zebulon nc,city_state.zelienople pa,city_state.zephyrhills fl,city_state.zimmerman mn,city_state.zionsville in
0,0.644279,0.605636,0.605636,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.578468,0.545663,0.545663,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.578490,0.562720,0.562720,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.578490,0.562720,0.562720,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.644239,0.583652,0.583652,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12813,0.061024,0.060949,0.059210,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12814,0.056295,0.056279,0.056279,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12815,0.111833,0.112107,0.112107,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12816,0.089104,0.067470,0.067470,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
